In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# the GPU id to use (0, 1, 2 or 3)
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import re
import glob
from keras.models import load_model
from keras.models import Model

from keras.layers import Input
from keras.layers import TimeDistributed

from keras import backend as K

In [5]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

In [6]:
def get_image(path, stats, rescale_method):
    img = np.load(path)
    img = img.astype('float32')
    return img

In [7]:
def get_gfp(path, stats, rescale_method, fit_method, gfp_thresh):
    gfp = np.load(path)
    if fit_method == 'classify':
        if gfp <= gfp_thresh:
            res = 0.
        else:
            res = 1.
    if fit_method == 'predict':
        if rescale_method == 'log':
            res = np.log(gfp)
        if rescale_method == 'log_center':
            res = np.log(gfp) - np.log(stats[2])
        if rescale_method == 'normalize':
            res = (gfp - stats[0])/(stats[1] - stats[0])
        if rescale_method == 'standardize':
            res = (gfp - stats[2])/stats[3]
        if rescale_method == 'log_normalize':
            res = (np.log(gfp) - np.log(stats[0]))/(np.log(stats[1]) - np.log(stats[0]))
        if rescale_method == 'log_standardize':
            res = (np.log(gfp) - np.log(stats[2]))/np.log(stats[3])
    return res

In [8]:
def rotate90xDegrees(img):
    ret =[]
    rows, cols = img.shape
    for i in range(1,4):
        M = cv2.getRotationMatrix2D((cols/2,rows/2),i*90,1)
        ret.append(cv2.warpAffine(img,M,(cols,rows)))
    return ret

In [9]:
def augment_image(img, aug_no):
    img_new = np.zeros((img.shape)).astype('float32')
    rows, cols, chans = img.shape
    for i in range(chans):
        if aug_no == 0:
            img_new[:, :, i] = img[:, :, i]
        if aug_no == 1:
            img_new[:, :, i] = np.flipud(img[:, :, i])
        if aug_no == 2:
            img_new[:, :, i] = np.fliplr(img[:, :, i])
        if aug_no == 3:
            ud = np.flipud(img[:, :, i])
            img_new[:, :, i] = np.fliplr(ud)
        if aug_no == 4:
            img_new[:, :, i] = rotate90xDegrees(img[:, :, i])[0]
        if aug_no == 5:
            ud = np.flipud(img[:, :, i])
            img_new[:, :, i] = rotate90xDegrees(ud)[0]
        if aug_no == 6:
            lr = np.fliplr(img[:, :, i])
            img_new[:, :, i] = rotate90xDegrees(lr)[0]
        if aug_no == 7:
            ud = np.flipud(img[:, :, i])
            udlr = np.fliplr(ud)
            img_new[:, :, i] = rotate90xDegrees(udlr)[0]
    
    return img_new

In [10]:
fold = 'fold1'

test_cell_dir = '/scratch-shared/phil/LNP/LNP_data_09/images_test'
test_gfp_dir = '/scratch-shared/phil/LNP/LNP_data_09/gfp_test'

wells = ['C04', 'C05', 'E03', 'E04', 'E05', 'G03', 'G04', 'G05']
n_wells = len(wells) 

tpoints = 'T0001'
for i in range(2, 10):
    tpoints = np.append(tpoints, 'T000' + str(i))
for i in range(10, 21):
    tpoints = np.append(tpoints, 'T00' + str(i))
n_time = len(tpoints)

all_cell = glob.glob(test_cell_dir + '/' '*')
all_cell = sorted(all_cell, key = natural_keys)
all_gfp = []
cell_well_tpoint = [] # well id and tpoint for each cell in each time point
for cell in all_cell:
    s0 = cell.split('test/')
    s1 = s0[1].split('_')
    f = glob.glob(test_gfp_dir + '/' + s1[0] + '_' + s1[1] + '_T0072_cell_' + s1[4])
    all_gfp.extend(f)
    cell_well_tpoint.append([s1[0], s1[2]])

fit_method = 'classify' # 'classify' or 'predict'
# previous explorations showed the following two rescaling methods to be the most appropriate
cell_rescale_method = 'standardize'
gfp_rescale_method = 'log_center'
cell_stats = np.load('/scratch-shared/phil/LNP/LNP_data_09/cell_stats.npy')
gfp_stats = np.load('/scratch-shared/phil/LNP/LNP_data_09/gfp_stats.npy')

latent_dim = 32

CNN = load_model('/scratch-shared/phil/LNP/LNP_data_09/CNN_' + fit_method + '_' + fold + '.h5')
encoder = Model(inputs=CNN.get_layer('input_1').output, outputs=CNN.get_layer('latent_vector').output)

LSTM = load_model('/scratch-shared/phil/LNP/LNP_data_09/LSTM_' + fit_method + '_' + fold + '.h5')

input_layer = Input(shape=(n_time, 192, 192, 3))
td_layer = TimeDistributed(encoder)(input_layer)
lstm_out = LSTM(td_layer)
model = Model(inputs=input_layer, outputs=lstm_out)

In [11]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 192, 192, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 192, 192, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 192, 192, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 96, 96, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 96, 96, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 96, 96, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 48, 48, 128)       0         
__________

In [12]:
set_trainable = False
for layer in encoder.layers:
    if layer.name == 'global_average_pooling2d_1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [13]:
LSTM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 32)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               49664     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 66,305
Trainable params: 66,305
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 192, 192, 3)   0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 32)            1744864   
_________________________________________________________________
model_1 (Model)              (None, 1)                 66305     
Total params: 1,811,169
Trainable params: 74,593
Non-trainable params: 1,736,576
_________________________________________________________________
